# Smart Paraphrasing Using Constrained Beam Search in NLP
Reference:
* https://towardsdatascience.com/smart-paraphrasing-using-constrained-beam-search-in-nlp-9af6fd046e5c
* code in https://colab.research.google.com/drive/1Xq7g8tTXDekL9DPKQXnhxsjCr7UqyUdT?usp=sharing
* further courses about question generation in NLP, referring to the last section of the blog

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

device  cpu


In [6]:
# Copy writing example - Use a given word in paraphrasing
context = "Supercharge your data science skills by building real world projects."
force_words = ["improve"]

text = "paraphrase: "+context + " </s>"
input_ids = tokenizer(text,max_length =128, padding=True, return_tensors="pt").input_ids
input_ids = input_ids.to(device)

# Beam search
outputs = model.generate(
    input_ids,
    num_beams=10,
    num_return_sequences=3,
    max_length=128,
    early_stopping=True,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)
print ("\nNormal beam search\n")
print ("Original: ",context)
for beam_output in outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

/Users/llv23/opt/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(



Normal beam search

Original:  Supercharge your data science skills by building real world projects.


2022-05-01 11:24:25.192802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.10.1.dylib


paraphrasedoutput: By implementing real world projects, you can improve your data science skills.
paraphrasedoutput: By implementing real world projects, you can boost your data science skills.
paraphrasedoutput: By implementing real world projects, you can enhance your data science skills.


In [7]:
# Constrained Beam search
force_words_ids = tokenizer(force_words, add_special_tokens=False).input_ids
outputs = model.generate(
    input_ids,
    force_words_ids=force_words_ids,
    max_length=128,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)
print ("\nConstrained beam search\n")
print ("Original: ",context)
for beam_output in outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)


Constrained beam search

Original:  Supercharge your data science skills by building real world projects.
paraphrasedoutput: By implementing real world projects, you can improve your data science skills.
paraphrasedoutput: By executing real world projects, you can improve your data science skills.
paraphrasedoutput: Build real world projects to improve your data science skills.
